In [108]:
import os
import librosa   #for audio processing
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from pedalboard import Pedalboard
from pedalboard.io import AudioFile
from IPython.display import Audio
from collections import defaultdict
import time
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import StringLookup
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer

AUTOTUNE = tf.data.experimental.AUTOTUNE

ImportError: cannot import name 'StringLookup' from 'tensorflow.keras.layers' (C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\tensorflow\keras\layers\__init__.py)

In [104]:
! pip install tensorflow --upgrade

Defaulting to user installation because normal site-packages is not writeable

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\cli\base_command.py", line 228, in _main
    status = self.run(options, args)
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\cli\req_command.py", line 182, in wrapper
    return func(self, options, args)
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\commands\install.py", line 323, in run
    requirement_set = resolver.resolve(
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\resolution\legacy\resolver.py", line 183, in resolve
    discovered_reqs.extend(self._resolve_one(requirement_set, req))
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\resolution\legacy\resolver.py", line 388, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)


  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\resolution\legacy\resolver.py", line 340, in _get_abstract_dist_for
    abstract_dist = self.preparer.prepare_linked_requirement(req)
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\operations\prepare.py", line 467, in prepare_linked_requirement
    local_file = unpack_url(
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\operations\prepare.py", line 255, in unpack_url
    file = get_http_url(
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\operations\prepare.py", line 129, in get_http_url
    from_path, content_type = _download_http_url(
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\operations\prepare.py", line 282, in _download_http_url
    for chunk in download.chunks:
  File "C:\Users\hrith\AppData\Roaming\Python\Python38\site-packages\pip\_internal\cli\progres

In [73]:
labels=os.listdir("data/")

In [74]:
labels

['increase_volume1.mp3',
 'increase_volume2.mp3',
 'next_song1.mp3',
 'next_song2.mp3',
 'pause_music1.mp3',
 'pause_music2.mp3',
 'play_spotify1.mp3',
 'play_spotify2.mp3',
 'processed_1000_increase_volume1.wav',
 'processed_1000_increase_volume2.wav',
 'processed_1000_next_song1.wav',
 'processed_1000_next_song2.wav',
 'processed_1000_pause_music1.wav',
 'processed_1000_pause_music2.wav',
 'processed_1000_play_spotify1.wav',
 'processed_1000_play_spotify2.wav',
 'processed_1000_reduce_volume1.wav',
 'processed_1000_reduce_volume2.wav',
 'processed_1000_skip_song1.wav',
 'processed_1000_skip_song2.wav',
 'processed_1000_stop_music1.wav',
 'processed_1000_stop_music2.wav',
 'processed_1001_increase_volume1.wav',
 'processed_1001_increase_volume2.wav',
 'processed_1001_next_song1.wav',
 'processed_1001_next_song2.wav',
 'processed_1001_pause_music1.wav',
 'processed_1001_pause_music2.wav',
 'processed_1001_play_spotify1.wav',
 'processed_1001_play_spotify2.wav',
 'processed_1001_reduce_

In [59]:
audio_types = {
    "increase_volume1.wav": "increase volume",
    "increase_volume2.wav": "increase volume",
    "next_song1.wav": "next song",
    "next_song2.wav": "next song",
    "pause_music1.wav": "pause music",
    "pause_music2.wav": "pause music",
    "play_spotify1.wav": "play spotify",
    "play_spotify2.wav": "play spotify",
    "reduce_volume1.wav": "reduce volume",
    "reduce_volume2.wav": "reduce volume",
    "skip_song1.wav": "skip song",
    "skip_song2.wav": "skip song",
    "stop_music1.wav": "stop music",
    "stop_music2.wav": "stop music",
}

In [79]:
all_file = []
all_label = []
counter = 0
for label in labels:
    if label[-4:] == ".wav":
        all_file.append(label[:-4])
        if counter % 1000 == 0:
            print(counter)
        for t in audio_types.keys():
            if (t in label):
                all_label.append(audio_types[t]) 
    counter += 1

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [86]:
metadata_df = pd.DataFrame({"file_name": all_file, "transcription": all_label})

In [93]:
df_train = metadata_df.sample(frac = 0.7)
rest_part_30 = metadata_df.drop(df_train.index)
df_dev = rest_part_30.sample(frac = 0.66)
df_test = rest_part_30.drop(df_dev.index)

In [107]:
# The set of characters accepted in the transcription.
characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]
# Mapping characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
# Mapping integers back to original characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

AttributeError: module 'tensorflow.keras.layers' has no attribute 'StringLookup'

In [101]:
# An integer scalar Tensor. The window length in samples.
frame_length = 256
# An integer scalar Tensor. The number of samples to step.
frame_step = 160
# An integer scalar Tensor. The size of the FFT to apply.
# If not provided, uses the smallest power of 2 enclosing frame_length.
fft_length = 384


def encode_single_sample(wav_file, label):
    ###########################################
    ##  Process the Audio
    ##########################################
    # 1. Read wav file
    file = tf.io.read_file('data/' + wav_file + ".wav")
    # 2. Decode the wav file
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    # 3. Change type to float
    audio = tf.cast(audio, tf.float32)
    # 4. Get the spectrogram
    spectrogram = tf.signal.stft(
        audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
    )
    # 5. We only need the magnitude, which can be derived by applying tf.abs
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    # 6. normalisation
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)
    ###########################################
    ##  Process the label
    ##########################################
    # 7. Convert label to Lower case
    label = tf.strings.lower(label)
    # 8. Split the label
    label = tf.strings.unicode_split(label, input_encoding="UTF-8")
    # 9. Map the characters in label to numbers
    label = char_to_num(label)
    # 10. Return a dict as our model is expecting two inputs
    return spectrogram, label

In [102]:
batch_size = 32
# Define the trainig dataset
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_train["file_name"]), list(df_train["transcription"]))
)
train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=AUTOTUNE)
)

# Define the develop dataset
develop_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_dev["file_name"]), list(df_dev["transcription"]))
)
develop_dataset = (
    develop_dataset.map(encode_single_sample, num_parallel_calls=AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=AUTOTUNE)
)

# Define the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_test["file_name"]), list(df_test["transcription"]))
)
test_dataset = (
    test_dataset.map(encode_single_sample, num_parallel_calls=AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=AUTOTUNE)
)

NameError: in user code:

    <ipython-input-101-a28a5c73175d>:40 encode_single_sample  *
        label = char_to_num(label)

    NameError: name 'char_to_num' is not defined
